In [2]:
import requests, re
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time, random
from urllib.request import urlopen
import urllib.parse
import numpy as np
from IPython.display import display

In [101]:
def import_df(path): # takes in file path of df
    try: 
        df = pd.read_csv(path, index_col=0)
        if 'overdrive_author' not in list(df.columns):
            df['overdrive_author'] = np.nan
        start_index = df[df.overdrive_author.isna()].iloc[0].name
#         print(start_index)
    except FileNotFoundError:
        return 0, 0
    
    return int(start_index), df

In [156]:
def import_df_redo(path): # takes in file path of df
    try: 
        df = pd.read_csv(path, index_col=0)
        if 'overdrive_author' not in list(df.columns):
            df['overdrive_author'] = np.nan
        sub_df = df[(df['Library copies'] == 'info unavailable') | (df.overdrive_author.isna())]
        indices = list(sub_df.index)
#         print(start_index)
    except FileNotFoundError:
        return 0, 0
    
    return indices, df

In [188]:
def get_author_OD(path):
    
    global driver_proxy
    driver_proxy = initiate_wbw()
    
    start_index, df = import_df(path)
    base_url = 'https://nypl.overdrive.com'
    


    position_start_index = list(df.index).index(start_index)
    list_of_indices = list(df.index[position_start_index:])
    
    
    for index in list_of_indices:
        
        
        if df.loc[index, 'overdrive_url'] != 'title not found':
#         print('ding')
            
#             print (base_url + df.loc[index, 'overdrive_url'])
#             break
    
    
            soup = get_soup_driver_proxy(base_url + df.loc[index, 'overdrive_url'])
        
#             try: cookie_button = driver_proxy.find_elements_by_xpath('//button[@aria-label="Holds ratio explanation."]')

            
        
        
#             if df.loc[index, 'overdrive_author'] == '-1':
            try: 
                overdrive_author = soup.find('a', class_='TitleDetailsHeading-creatorLink').text
                df.loc[index, 'overdrive_author'] = overdrive_author
            except Exception as e:
                overdrive_author = -1
                print(e)



#             df.loc[index, 'overdrive_author'] = overdrive_author      

            if df.loc[index, 'People waiting per copy'] == 'info unavailable':
                
                            # check for a holds_info button. If there, click it.
                try:

                    # click hold_time_info button
                    buttons = driver_proxy.find_elements_by_xpath('//button[@aria-label="Holds ratio explanation."]')
                    buttons[0].click()
                    time.sleep(3 + random.uniform(-1,1))

                    # grab html and parse into soup
                    soup = bs(driver_proxy.page_source, 'html.parser') 

                    # extract hold_info like wait-time, number of people waiting, etc.
                    holds_info_soup = soup.find_all('p', class_='detail')
                    holds_info = [info.text.split(':') for info in holds_info_soup]

                    # remove extra spaces
                    list_results = []
                    for row in holds_info:
                        row[:] = [info.strip() for info in row]

                    # convert result into sub_df
                    holds_info = np.array(holds_info)
                    sub_df = pd.DataFrame(holds_info.T, columns = holds_info[:,0]) ### sub_df might only be partially formed         
                    sub_df.drop([0], inplace=True)

                    # if holds_info contains incomplete holds_info, raise Error to remove data point
    #                 print(sub_df.columns)
    #                 print(list(sub_df.columns) != ['Wait time', 'Library copies', 'People waiting in total', 'People waiting per copy'])
                    if list(sub_df.columns) != ['Wait time', 'Library copies', 'People waiting in total', 'People waiting per copy']:
                        raise Exception('column names don\'t match')

    #                 print('checked for holds_info button')
    #                 display(sub_df)


                # if no holds_info button, it may be because copies are available. Check.
                except:
    #                 print('apparently there was an error checking for holds_button button. maybe missing col names or no holds button')
                    # grab html and parse into soup
                    soup = bs(driver_proxy.page_source, 'html.parser')

                    # yes copies are available and therefore no holds_info button
                    try:
                        # there is no holds_info button so check for borrow button
                        if soup.find('button', class_='TitleActionButton').text == 'Borrow':
                            copies_avail = soup.find('button', class_='TitleActionButton').parent.parent.parent.parent.find(class_='availabilityText').text.strip().split()
                            if copies_avail[0] == 'Always':
                                holds_info = np.array([['Wait time', 'Library copies', 'People waiting in total', 'People waiting per copy'],
                                         [0, 'Always available', 'Unknown', 0]])    
                            else:    
                                holds_info = np.array([['Wait time', 'Library copies', 'People waiting in total', 'People waiting per copy'],
                                         [0, copies_avail[2], str(-int(copies_avail[0])), 0]])
    #                         print('there is a borrow button')
                            sub_df = pd.DataFrame(holds_info, columns = holds_info[0]) 
                            sub_df.drop([0], inplace=True)

                        else:
    #                         print('there is no borrow button')
                            raise Exception('no borrow button')

                    # no copies available and no holds_info button
                    except:
    #                     print('no copies available and no holds_info button')
                        holds_info = np.array([['Wait time', 'Library copies', 'People waiting in total', 'People waiting per copy'],
                                     ['info unavailable', 'info unavailable', 'info unavailable', 'info unavailable']])

                        sub_df = pd.DataFrame(holds_info, columns = holds_info[0]) 
                        sub_df.drop([0], inplace=True)

#                 display(sub_df)
                df.loc[index, 'Wait time'] = sub_df.loc[1, 'Wait time']
                df.loc[index, 'Library copies'] = sub_df.loc[1, 'Library copies']
                df.loc[index, 'People waiting in total'] = sub_df.loc[1, 'People waiting in total']
                df.loc[index, 'People waiting per copy'] = sub_df.loc[1, 'People waiting per copy']
        elif df.loc[index, 'People waiting per copy'] == 'title not found':
            df.loc[index, 'overdrive_author'] = -1
        else:
            df.loc[index, 'overdrive_author'] = 'unknown error in get_author_OD'
        df.to_csv(path)
        time.sleep(10 + random.uniform(-1,1))
        
#     return pd.DataFrame(list_results)
    

In [182]:
def get_author_OD_redo(path):
    
    global driver_proxy
    driver_proxy = initiate_wbw()
    
    list_of_indices, df = import_df_redo(path)
    base_url = 'https://nypl.overdrive.com'
    


#     position_start_index = list(df.index).index(start_index)
#     list_of_indices = list(df.index[position_start_index:])
    
    
    for index in list_of_indices:
        
        index = int(index)
        if df.loc[index, 'overdrive_url'] != 'title not found':
#         print('ding')
            
#             print (base_url + df.loc[index, 'overdrive_url'])
#             break
    
    
            soup = get_soup_driver_proxy(base_url + df.loc[index, 'overdrive_url'])
        

            
        
        
            if df.loc[index, 'overdrive_author'] == '-1':
                try: overdrive_author = soup.find('a', class_='TitleDetailsHeading-creatorLink').text
                except: overdrive_author = -1
            


#             df.loc[index, 'overdrive_author'] = overdrive_author      

            if df.loc[index, 'People waiting per copy'] == 'info unavailable':
                
                            # check for a holds_info button. If there, click it.
                
                
                try:

                    # click hold_time_info button
                    buttons = driver_proxy.find_elements_by_xpath('//button[@aria-label="Holds ratio explanation."]')
                    buttons[0].click()
                    time.sleep(3 + random.uniform(-1,1))
                    
                    try: 
                        cookie_button = driver_proxy.find_elements_by_xpath('//button[@class="confirm button radius margin-small secondary right"]')
                        cookie_button.click()
                        # let page load
#                         time.sleep(2 + random.uniform(-1,0))
                        buttons = driver_proxy.find_elements_by_xpath('//button[@aria-label="Holds ratio explanation."]')
                        buttons[0].click()
#                         time.sleep(3 + random.uniform(-1,1))
                    except:
                        pass
        
        
        

                    # grab html and parse into soup
                    soup = bs(driver_proxy.page_source, 'html.parser') 

                    # extract hold_info like wait-time, number of people waiting, etc.
                    holds_info_soup = soup.find_all('p', class_='detail')
                    holds_info = [info.text.split(':') for info in holds_info_soup]

                    # remove extra spaces
                    list_results = []
                    for row in holds_info:
                        row[:] = [info.strip() for info in row]

                    # convert result into sub_df
                    holds_info = np.array(holds_info)
                    sub_df = pd.DataFrame(holds_info.T, columns = holds_info[:,0]) ### sub_df might only be partially formed         
                    sub_df.drop([0], inplace=True)

                    # if holds_info contains incomplete holds_info, raise Error to remove data point
    #                 print(sub_df.columns)
    #                 print(list(sub_df.columns) != ['Wait time', 'Library copies', 'People waiting in total', 'People waiting per copy'])
                    if list(sub_df.columns) != ['Wait time', 'Library copies', 'People waiting in total', 'People waiting per copy']:
                        raise Exception('column names don\'t match')
                     
    #                 print('checked for holds_info button')
#                         #                 display(sub_df)
#                         try:  
#                             df.loc[index, 'Wait time'] = sub_df.loc[1, 'Wait time']
#                         except:
#                             pass                        
#                         try:
#                             df.loc[index, 'Library copies'] = sub_df.loc[1, 'Library copies']
#                         except:
#                             pass
#                         try:  
#                             df.loc[index, 'People waiting in total'] = sub_df.loc[1, 'People waiting in total']
#                         except:
#                             try:
#                                 df.loc[index, 'People waiting in total'] = sub_df.loc[1, 'People waiting']
#                             except:
#                                 pass
#                         try:  
#                             df.loc[index, 'People waiting per copy'] = sub_df.loc[1, 'People waiting per copy']
#                         except:
#                             pass

                # if no holds_info button, it may be because copies are available. Check.
                except:
    #                 print('apparently there was an error checking for holds_button button. maybe missing col names or no holds button')
                    # grab html and parse into soup
                    soup = bs(driver_proxy.page_source, 'html.parser')

                    # yes copies are available and therefore no holds_info button
                    try:
                        # there is no holds_info button so check for borrow button
                        if soup.find('button', class_='TitleActionButton').text == 'Borrow':
                            copies_avail = soup.find('button', class_='TitleActionButton').parent.parent.parent.parent.find(class_='availabilityText').text.strip().split()
                            if copies_avail[0] == 'Always':
                                holds_info = np.array([['Wait time', 'Library copies', 'People waiting in total', 'People waiting per copy'],
                                         [0, 'Always available', 'Unknown', 0]])    
                            else:    
                                holds_info = np.array([['Wait time', 'Library copies', 'People waiting in total', 'People waiting per copy'],
                                         [0, copies_avail[2], str(-int(copies_avail[0])), 0]])
    #                         print('there is a borrow button')
                            sub_df = pd.DataFrame(holds_info, columns = holds_info[0]) 
                            sub_df.drop([0], inplace=True)

                        else:
    #                         print('there is no borrow button')
                            raise Exception('no borrow button')

                    # no copies available and no holds_info button
                    except:
    #                     print('no copies available and no holds_info button')
                        holds_info = np.array([['Wait time', 'Library copies', 'People waiting in total', 'People waiting per copy'],
                                     ['info unavailable', 'info unavailable', 'info unavailable', 'info unavailable']])

                        sub_df = pd.DataFrame(holds_info, columns = holds_info[0]) 
                        sub_df.drop([0], inplace=True)

#                 display(sub_df)
                df.loc[index, 'Wait time'] = sub_df.loc[1, 'Wait time']
                df.loc[index, 'Library copies'] = sub_df.loc[1, 'Library copies']
                df.loc[index, 'People waiting in total'] = sub_df.loc[1, 'People waiting in total']
                df.loc[index, 'People waiting per copy'] = sub_df.loc[1, 'People waiting per copy']
        elif df.loc[index, 'People waiting per copy'] == 'title not found':
            df.loc[index, 'overdrive_author'] = -1
        else:
            df.loc[index, 'overdrive_author'] = 'unknown error in get_author_OD'
        df.to_csv(path)
        time.sleep(10 + random.uniform(-1,1))
        
#     return pd.DataFrame(list_results)
    

In [3]:
def initiate_wbw(): # launches headless browser selenium
    options = Options()
    options.headless = False
    return webdriver.Chrome(options=options)

In [177]:
def get_soup_driver_proxy(base_url): # Retrieves soup from website using Selenium
#     driver = webdriver.Chrome()
    YOUR_API_KEY = 'OavaC96DqSNsO1vM2qStOhzTZoeOakNd'
    target_url = urllib.parse.quote(base_url, safe='~')

    URL = f"https://api.webscrapingapi.com/v1/?api_key={YOUR_API_KEY}&url=" + target_url

    driver_proxy.get(URL)
    time.sleep(2 + random.uniform(-1,1))

    # check if accept cookie button

    try: 
        cookie_button = driver_proxy.find_elements_by_xpath('//button[@class="confirm button radius margin-small secondary right"]')
        cookie_button[0].click()
        time.sleep(3 + random.uniform(-1,0))
#         driver_proxy.get(URL)

    except:
        pass
    
    
    r = driver_proxy.page_source
    
#     driver.quit()
    return bs(r, 'html.parser')  
#     return r

In [146]:
global driver_proxy
driver_proxy = initiate_wbw()

In [138]:
driver_proxy.get('https://nypl.overdrive.com/media/599788')

In [149]:
soup = get_soup_driver_proxy('https://nypl.overdrive.com/media/3783541')

In [151]:
driver_proxy.quit()

In [186]:
def scrape(path):
    error_counter = 0
    try: get_author_OD(path)
    except:
        print('error')
        time.sleep(60 + random.uniform(-2,2))
        error_counter += 1
        if error_counter > 3:
            return 'Terminated, check for errors'
        scrape(path)

In [123]:
def new_scrape_redo(path, error_counter=0):
    error_counter = error_counter
    try: get_author_OD_redo(path)
    except:
        print('error')
        time.sleep(60 + random.uniform(-2,2))
        error_counter += 1
        if error_counter > 2:
            return 'Terminated, check for errors'
        new_scrape_redo(path, error_counter)

In [189]:
scrape('Figures/get_author_OD.csv')

In [ ]:
path = 
test_df = pd.read_csv(path, index_col=0)

